In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

In [ ]:
def get_soup(url):
    t = 0
    if url.startswith("https://www.amazon.com/dp/") or url.startswith("https://www.amazon.com/s?i"):
        time_wait = 20
    else:
        time_wait = 2
    while True:
        r = requests.get("http://localhost:8050/render.html", params={"url": url,'wait': time_wait, 'viewport': '320x480', 'images': 0, 'resource_timeout': 10})
        soup = BeautifulSoup(r.text, "html.parser")
        if soup.text.find('GlobalTimeoutError') != -1:
            time.sleep(31)
            print(f"time sleep try {t}")
            time_wait += 4
            if t == 2:
                print(f"too many timeouts (2) for url: {url}")
                break
            t += 1
        else:
            print(f"page successfully loaded in time {time_wait}: {url}")
            break       
    return soup

In [ ]:
all_product_links = []
a = 0
i = 100
with open(f"../../gen/scraping/products/amazon_links.csv", "a", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    while True:
        page_link = f"https://www.amazon.com/s?i=mobile&bbn=7072561011&rh=n%3A7072561011%2Cp_n_availability%3A2661601011%2Cp_n_feature_twenty_browse-bin%3A17881876011%7C17881878011%7C17881879011&dc&fs=true&page={i}&qid=1665060482&rnid=17881875011&ref=sr_pg_{i}"
        #try:
        soup = get_soup(page_link)
        print(f"currently looking at page {i}")
        if soup.title.text=="Sorry! Something went wrong!":
            print("Blocked")
            break
        else:
            links = soup.find_all(class_="a-link-normal s-no-outline")
        a -= 1
        for link in links:
            if link['href'].find('/dp/') != -1 and not link['href'].startswith('/dp/'):
                link_asin = get_link_asin(link)
                all_product_links.append(link_asin)
                writer.writerow(list(link_asin.values()))
        if a < 0:
            a = 0
        elif a == 5:
            break
        try:
            soup.find('span', class_ = "s-pagination-item s-pagination-next s-pagination-disabled").text
            print('last page')
            break
        except:
            pass
        print(len(all_product_links))
        i += 1